In [5]:
# Week1 HW: Polynomial Regression
####ESC 2020 - 1 신혜연

import numpy as np 
np.set_printoptions(precision=3)
import pandas as pd 
pd.set_option('display.precision',3)
import matplotlib.pyplot as plt
import seaborn as sns

### Importing the dataset
bike = pd.read_csv('C:/Users/USER/Desktop/[2020]/ESC/day.csv')
bike # 731개에 16개 특성 지닌 것 확인.

### Data Na check
bike.info() # No NA value 

Data = bike[['cnt', 'temp']]
Data.describe()

Data.hist(bins=50, figsize=(12, 5))
plt.tight_layout()
sns.pairplot(Data)

##Polynomial Regression

data = Data.sample(n=30).reset_index() 
#### -> 이걸 왜 30개만 뽑았을까? 생각해보기.
###### 생각을 해보다가 구글의 힘을 빌렸는데 표본 크기를 m이라고 했을 때, 보통 회귀에서 설명변수의 갯수는 m/10 또는 m/20보다 작아야 된다고 합니다(Harell, 2001, p.61). 설명 변수의 분포에 의존하여, 만약 설명변수의 범위가 넓다면 설명 변수의 갯수는 m/20보다 작아야 하며, 그렇지 않다면(범주변수의 경우에는 드문 범주의 경우) m/10보다 작아야 좋다고 하네요!
###### cf. "https://snucss.wordpress.com/2016/03/26/%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D%EC%97%90%EC%84%9C-%EC%A0%81%EC%A0%88%ED%95%9C-%ED%91%9C%EB%B3%B8-%ED%81%AC%EA%B8%B0/"
###1) Train-Test Split

N= data.shape[0]
test = np.random.choice(np.arange(N),20, replace=False)
train_df = data[~data.index.isin(test)].copy()
test_df = data[data.index.isin(test)].copy()
print(train_df.shape, test_df.shape)

y = np.matrix(data['cnt']).T
X = np.matrix(data['temp']).T
y_train = np.matrix(train_df['cnt']).T
X_train = np.matrix(train_df['temp']).T
y_test = np.matrix(test_df['cnt']).T
X_test = np.matrix(test_df['temp']).T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
instant       731 non-null int64
dteday        731 non-null object
season        731 non-null int64
yr            731 non-null int64
mnth          731 non-null int64
holiday       731 non-null int64
weekday       731 non-null int64
workingday    731 non-null int64
weathersit    731 non-null int64
temp          731 non-null float64
atemp         731 non-null float64
hum           731 non-null float64
windspeed     731 non-null float64
casual        731 non-null int64
registered    731 non-null int64
cnt           731 non-null int64
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB
(10, 3) (20, 3)


In [6]:
# Week1 HW: Polynomial Regression
####ESC 2020 - 1 신혜연

import numpy as np 
np.set_printoptions(precision=3)
import pandas as pd 
pd.set_option('display.precision',3)
import matplotlib.pyplot as plt
import seaborn as sns

### Importing the dataset
bike = pd.read_csv('C:/Users/USER/Desktop/[2020]/ESC/day.csv')
bike # 731개에 16개 특성 지닌 것 확인.

### Data Na check
bike.info() # No NA value 

Data = bike[['cnt', 'temp']]
Data.describe()

Data.hist(bins=50, figsize=(12, 5))
plt.tight_layout()
sns.pairplot(Data)

##Polynomial Regression

data = Data.sample(n=30).reset_index() 
#### -> 이걸 왜 30개만 뽑았을까? 생각해보기.
###### 생각을 해보다가 구글의 힘을 빌렸는데 표본 크기를 m이라고 했을 때, 보통 회귀에서 설명변수의 갯수는 m/10 또는 m/20보다 작아야 된다고 합니다(Harell, 2001, p.61). 설명 변수의 분포에 의존하여, 만약 설명변수의 범위가 넓다면 설명 변수의 갯수는 m/20보다 작아야 하며, 그렇지 않다면(범주변수의 경우에는 드문 범주의 경우) m/10보다 작아야 좋다고 하네요!
###### cf. "https://snucss.wordpress.com/2016/03/26/%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D%EC%97%90%EC%84%9C-%EC%A0%81%EC%A0%88%ED%95%9C-%ED%91%9C%EB%B3%B8-%ED%81%AC%EA%B8%B0/"
###1) Train-Test Split

N= data.shape[0]
test = np.random.choice(np.arange(N),20, replace=False)
train_df = data[~data.index.isin(test)].copy()
test_df = data[data.index.isin(test)].copy()
print(train_df.shape, test_df.shape)

y = np.matrix(data['cnt']).T
X = np.matrix(data['temp']).T
y_train = np.matrix(train_df['cnt']).T
X_train = np.matrix(train_df['temp']).T
y_test = np.matrix(test_df['cnt']).T
X_test = np.matrix(test_df['temp']).T

###2) Design Matrix, Linear Regression Fitting = test, train log 씌운 MSE 값 뱉는 fct 구하기
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error as mse

trainmse = []
testmse = []
for i in range(1,13):
    poly = PolynomialFeatures(degree=i)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)
    
    lin = LinearRegression(fit_intercept=False) #linear regression fitting
    lin.fit(X_train_poly, y_train) #fit한 모델 탄생
    
    test_mse =  np.log(mse(y_train, lin.predict(X_test_poly)))
    testmse.append(test_mse)

    train_mse =  np.log(mse(y_train, lin.predict(X_train_poly))
    trainmse.append(train_mse)

                        
###3) train , test mse 나타내는 그래프를 그리자
x = range(1,13)
plt.plot(x,testmse)
plt.plot(x,trainmse)

plt.xlabel("Degree of polynomial")
plt.xlabel("MSE")
plt.legend(['test mse','train mse'])
plt.show()

SyntaxError: invalid syntax (<ipython-input-6-3f3ccd8e7c8c>, line 65)

In [ ]:
######??? 왜 저기 오류났는지 모르는 상태에서 우선 숙제 제출합니다. 